## Exercise 1
Some simple plotting

## Exercise 2
Some more advanced plotting

## Exercise 3
- Computing OLS by matrix inversion
- Plotting OLS and data
- Comparing to scipy regression function

## Exercise 4
Some optimization problem

## Exercise 5 -> ADD QUESTION FROM CLIMATE_ECON PS 4 

Solve the model in the previous question (without the externality) numerically for a more general setting, in particular with a CIES utility function and less than full depreciation. Choose reasonable values for the parameters. How does the optimal extraction profile depend on the growth rate of total factor productivity?

### Answer
Solving the model numerically is straightforward. The code should look familiar from the two-period model we have seen in previous lectures. In particular, we can again use Scipy's minimization routine (**scipy.optimize.minimize**).

Note that since there is no extraction or externality cost caused by fossil fuel use, the complete resource stock will be used over the two periods.

In [ ]:
## Define functions
def u(x, nu):
    """
    Utility function
    """
    if nu == 1: # logarithmic utility
        return np.log(x)
    else:
        return (1 - nu)**(-1) * x**(1-nu)
    
def F(k, e, alpha, A):
    """
    Production function: Cobb-Douglas
    """
    return A * k**(alpha) * e**(1-alpha)

def obj(x, alpha, A1, g, beta, delta, nu, K1):
    """
    Computes the social planner's objective function, i.e. the representative agent's lifetime utility
    """
    E1, I = x
    return - u(F(K1, E1, alpha, A1) - I, nu) \
            - beta * u( F( (1-delta) * K1 + I, R - E1, alpha, (1 + g) * A1) + (1 - delta) * ( (1-delta) * K1 + I), nu)


## Set parameters
alpha = 0.6
beta = 0.98
R = 1
K1 = 1
A1 = 1
nu = 2
delta = 0.9

## Closed-form solution (with log utility and full depreciation), used as initial guess below
E1 = R * (1 + alpha * beta)/(1 + alpha * beta + beta)
K2 = alpha * beta * F(K1, E1, alpha, A1) / (1 + alpha * beta)
x0 = [E1, K2]
print(x0)

## determine bounds for choice variables
bnds = [(1e-8, 1e+12), (1e-8, 1e+12)]


## Baseline
g = 0.1
# run minimization routine
res = scipy.optimize.minimize(obj, x0,\
                              args = tuple([alpha, A1, g, beta, delta, nu, K1]), \
                              bounds = bnds, method='SLSQP', tol = 1e-12)
print(res.x)

## Increase productivity growth
g = 0.2
res = scipy.optimize.minimize(obj, x0,\
                              args = tuple([alpha, A1, g, beta, delta, nu, K1]), \
                              bounds = bnds, method='SLSQP', tol = 1e-12)
print(res.x)

We can see that less than full depreciation and non-logarithmic utility, higher productivity growth leads to less investment and hence a lower capital stock in period 2. This is intuitive: recall that $\nu > 1$ indicates a lower elasticity of intertemporal substitution (or a higher aversion to intertemporal consumption inequality), hence if the household is richer in the future (due to a higher productivity), the incentive to save today is weaker. A lower capital stock in period 2 implies that the marginal return to capital - $F_K(K_2, E_2) + 1 - \delta$ - increases. By the Hotelling rule, this implies that the *relative* marginal return to fossil fuel in period 2 (relative to period 1) has to rise as well. This can be achieved by shifting extraction from period 2 to period 1, thus leading to a (albeit small) increase in current fossil fuel use.